In [ ]:
include("../load.jl")

In [ ]:
using CSV, DataFrames, Statistics, Random

In [ ]:
using Distributed
addprocs(5)

In [ ]:
columns = ["Re", "thick", "M", "C_L"];
X = CSV.read(OCT.DATA_DIR * "airfoil/airfoil_X.csv", DataFrame, copycols=true, header=columns, delim=",");
Y = CSV.read(OCT.DATA_DIR * "airfoil/airfoil_Y.csv", DataFrame, copycols=true, header=["C_D"], delim=",");
# Re = Array(range(10000, stop=35000, step=5000));
# thick = [100,110,120,130,140,145];
# M = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9];
# cl = Array(range(0.35, stop=0.70, step=0.05));

In [ ]:
# Plotting some perspectives on the data
using Plots
plt = Plots.plot(X[:,3], X[:,4], Y[:,1], seriestype=:scatter, markersize = 2)
display(plt)

In [ ]:
m = JuMP.Model(with_optimizer(Gurobi.Optimizer))
gm = GlobalModel(model = m)
add_variables_from_data!(gm, X) # Adding variables for each data column.
add_variables_from_data!(gm, Y)
bound_to_data!(gm, X)           # Making sure we bound both our free variables and dependent variables 
bound_to_data!(gm, Y)           # test points and observations. 
add_datadriven_constraint(gm, X, Y.C_D, name = "drag polar", dependent_var = gm.model[:C_D])
gm("drag polar")

In [ ]:
learn_constraint!(gm, max_depth = 5, hyperplane_config = (sparsity=1,))
gm("drag polar").learners[end]

In [ ]:
add_tree_constraints!(gm)
gm.bbls[1].mi_constraints